---
# **Модуль 1. Python**
---

### **В данном модуле рассматриваются:**
- **Python Quick Recap**
- **Инструменты Python для работы с сетевым оборудованием**
- **Инструменты Python для работы с файлами конфигурации**
- **Nornir**

---

### **Python Quick Recap**

Переменные и типы данных

In [3]:
a = 10
b = 'text'
c = [1, 2, 3, 4]
d = (a, b, c)
e = {'name': 'device_1', 'ip_address': '10.78.3.230'}
f = {1, 2, 3, 5, 5}

In [4]:
print(f)

{1, 2, 3, 5}


Условия

In [5]:
if a == 10:
    print(f'a is equal to {a}')

a is equal to 10


In [6]:
if b in 'data':
    print(b)
elif b in 'sample':
    print(b[1:3])
else:
    print(list(b))

['t', 'e', 'x', 't']


Циклы

In [ ]:
f = 5

while f:
    print('f is not equal to 0')
    f-=1
else:
    print('f became 0')

In [7]:
for i in range(5):
    print(f'10.78.230.{i}')

10.78.230.0
10.78.230.1
10.78.230.2
10.78.230.3
10.78.230.4


In [ ]:
for i in c:
    if i == 3:
        break
    print(i)
else:
    print('End')

Функции

In [ ]:
def func1():
    pass

In [8]:
def func2(a, b):
    return a+b

print(func2(10, 20))

30


In [ ]:
def func3(name='localhost', address='127.0.0.1'):
    print(name, address)
    
func3(address='192.168.0.1')

Модули

In [10]:
import time
from sys import version

print(version)
time.sleep(2)
print('2 seconds after')

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
2 seconds after


---

### **Инструменты Python для работы с сетевым оборудованием**

### **Библиотека ```Netmiko```**

In [11]:
!pip install netmiko

Defaulting to user installation because normal site-packages is not writeable


In [12]:
from netmiko import Netmiko

---
Для осуществления подключения к устройствам используются классы ```Netmiko``` или ```ConnectHandler```. В качестве аргументов для инициализации экземпляров классов подаются параметры устройства в виде словаря или в виде ключевых аргументов. Среди поддерживаемых типов устройств представлены:

 - Cisco (IOS, IOS-XE, IOS-XR, NX-OS)
 - JunOS
 - Arista
 - F5
 - Checkpoint
 - Ruckus

In [13]:
leaf1_test = {
    "host": "172.20.20.11", 
    "username": "admin",
    "password": "admin",
    "device_type": "arista_eos"
}

In [14]:
leaf1 = Netmiko(**leaf1_test)

In [ ]:
leaf1.

---
Одним из основных методов данного класса является метод ```send_command()```, который принимает в качестве аргумента строку с командой для выполнения. Результат вызова метода присваивается переменной, в которую помещается отклик устройства на ввод команды.

In [15]:
output = leaf1.send_command('show version')

print(output)

Arista cEOSLab
Hardware version: 
Serial number: 73AE505E46FD5F8A965B00DEEBC3C3A7
Hardware MAC address: 001c.7322.f5b3
System MAC address: 001c.7322.f5b3

Software image version: 4.28.3M-28837868.4283M (engineering build)
Architecture: x86_64
Internal build version: 4.28.3M-28837868.4283M
Internal build ID: e950541d-fc75-4ced-a669-6fd2007c17b7
Image format version: 1.0
Image optimization: None

cEOS tools version: 1.1
Kernel version: 6.2.0-26-generic

Uptime: 19 minutes
Total memory: 16368668 kB
Free memory: 10830744 kB



---
Если необходимо выполнить несколько команд подряд, то последовательно вызывается ряд функций ```send_command()``` или метод ```send_config_set()```, в который подается список команд (в случае Cisco устройств в данном случае автоматически отправляется команда `configure terminal`).

In [17]:
leaf1.enable()

commands = ['ip routing', 'interface Loopback100', 'ip add 11.11.11.11/32']

output = leaf1.send_config_set(commands)

print(output)

config term
leaf1(config)#ip routing
leaf1(config)#interface Loopback100
ip add 11.11.11.11/32
leaf1(config-if-Lo100)#ip add 11.11.11.11/32
leaf1(config-if-Lo100)#end
leaf1#


---
Аналогично используется метод ```send_config_from_file()```, которая отправляет команды из указанного файла.

In [18]:
leaf1.enable()

output = leaf1.send_config_from_file('files/commands.txt')

print(output)

config term
leaf1(config)#show lldp neighbor
Last table change time   : 0:26:52 ago
Number of table inserts  : 2
Number of table deletes  : 0
Number of table drops    : 0
Number of table age-outs : 0

Port          Neighbor Device ID       Neighbor Port ID    TTL
---------- ------------------------ ---------------------- ---
Et1           leaf2                    Ethernet1           120
Ma0           leaf2                    Management0         120

leaf1(config)#
leaf1(config)#end
leaf1#


---
Методы ```config_mode()``` и ```exit_config_mode()``` позволяют соотвественно перевести устройство в конфигурационный режим и выйти из него. Метод ```find_prompt()``` возвращает текущую подсказку режима.

In [19]:
leaf1.config_mode()

print(leaf1.find_prompt())

leaf1.exit_config_mode()

print(leaf1.find_prompt())

leaf1(config)#
leaf1#


---
Для завершения сеанса используется метод ```disconnect()```. Метод ```save_config()```, сохраняет конфигурацию способом по умолчанию для данного вида устройств.

In [ ]:
leaf1.disconnect()

---
4.8 Для сетевых устройств Juniper и Cisco IOS-XR определен метод ```send_commit()```, который используется для подтверждения изменений конфигурации.

---
Для корректного обязательного завершения сеанса рекомендуется комбинировать методы библиотеки ```netmiko``` с использованием оператора ```with ... as ...```. 

In [20]:
leaf1_test = {
    "host": "172.20.20.11",
    "username": "admin",
    "password": "admin",
    "device_type": "arista_eos",
}

In [21]:
commands = ['show ip route']

In [22]:
with Netmiko(**leaf1_test) as leaf1:
    leaf1.enable()
    output = leaf1.send_config_set(commands)
    print(output)

config term
leaf1(config)#show ip route

VRF: default
Codes: C - connected, S - static, K - kernel, 
       O - OSPF, IA - OSPF inter area, E1 - OSPF external type 1,
       E2 - OSPF external type 2, N1 - OSPF NSSA external type 1,
       N2 - OSPF NSSA external type2, B - Other BGP Routes,
       B I - iBGP, B E - eBGP, R - RIP, I L1 - IS-IS level 1,
       I L2 - IS-IS level 2, O3 - OSPFv3, A B - BGP Aggregate,
       A O - OSPF Summary, NG - Nexthop Group Static Route,
       V - VXLAN Control Service, M - Martian,
       DH - DHCP client installed default route,
       DP - Dynamic Policy Route, L - VRF Leaked,
       G  - gRIBI, RC - Route Cache Route

Gateway of last resort:
 S        0.0.0.0/0 [1/0] via 172.20.20.1, Management0

 C        11.11.11.11/32 is directly connected, Loopback100
 C        172.20.20.0/24 is directly connected, Management0

leaf1(config)#end
leaf1#


In [23]:
help(leaf1.send_command)

Help on method send_command in module netmiko.base_connection:

send_command(command_string, expect_string=None, delay_factor=1, max_loops=500, auto_find_prompt=True, strip_prompt=True, strip_command=True, normalize=True, use_textfsm=False, use_genie=False) method of netmiko.arista.arista.AristaSSH instance
    Execute command_string on the SSH channel using a pattern-based mechanism. Generally
    used for show commands. By default this method will keep waiting to receive data until the
    network device prompt is detected. The current network device prompt will be determined
    automatically.
    
    :param command_string: The command to be executed on the remote device.
    :type command_string: str
    
    :param expect_string: Regular expression pattern to use for determining end of output.
        If left blank will default to being based on router prompt.
    :type expect_string: str
    
    :param delay_factor: Multiplying factor used to adjust delays (default: 1).
    :ty

### **Библиотека ```napalm```**

---
Библиотека ```napalm``` (Network Automation and Programmability Abstraction Layer with Multivendor support) реализует набор функций для взаимодействия с различными сетевыми операционными системами, используя унифицированный API. Поддерживаются следующие операционные системы:

 - Arista EOS
 - Cisco IOS
 - Cisco IOS-XR
 - Cisco NX-OS
 - Juniper JunOS
 - ...
 
Для начала работы с ```napalm``` необходимо установить саму библиотеку и зависимые пакеты:

```linux
pip install napalm

sudo apt-get install -y --force-yes libssl-dev libffi-dev python-dev python-cffi
```

---
Для проверки возможности работы с сетевым устройством может быть использован класс ```get_network_driver```, конструктор которого создает инструмент подключения для указанной сететвой операционной системы. Метод ```open()``` используется для вызова подключения к устройству.

In [ ]:
!pip install napalm

In [ ]:
from napalm import get_network_driver
from pprint import pprint

In [ ]:
import pyeapi

def patch_pyeapi_ciphers():
    """
    python -c 'import ssl; print(ssl._DEFAULT_CIPHERS)'
    @SECLEVEL=2:ECDH+AESGCM:ECDH+CHACHA20:ECDH+AES:DHE+AES:!aNULL:!eNULL:!aDSS:!SHA1:!AESCCM
    """
    try:
        import pyeapi.eapilib
    except ImportError:
        return

    connect_orig = pyeapi.eapilib.HttpsConnection.connect

    def connect(self):
        self._context.set_ciphers('DEFAULT@SECLEVEL=2')
        return connect_orig(self)

    pyeapi.eapilib.HttpsConnection.connect = connect


patch_pyeapi_ciphers()

In [ ]:
driver = get_network_driver('eos')
leaf1 = driver('172.20.20.11', 'admin', 'admin', optional_args = {'eos_autoComplete': True})

In [ ]:
leaf1.open()

---
Стандартные методы и аттрибуты экземпляра класса ```dirver()```. Позволяют как получать информацию по сетевому устройству, так и настраивать его. Для просмотра основной информации по устройству может быть вызван метод ```get_facts()```.

In [ ]:
pprint(leaf1.get_facts())

>Не все методы реализованы для всех поддерживаемых устройств. В случае, если метод не может быть вызван для выбранного устройства, вызывается исключение ```NotImplementedError```. В документации к библиотеке сказано, что, возможно, некоторые методы могут быть не включены в стандартный набор вообще, т.е. потенциально они могут быть доработаны и реализованы самостоятельно. 

---
Для получения информации по интерфейсам используются методы ```get_interfaces()```, ```get_interfaces_counters``` и ```get_interfaces_ip```.

In [ ]:
pprint(leaf1.get_interfaces())

In [ ]:
pprint(leaf1.get_interfaces_counters())

In [ ]:
pprint(leaf1.get_interfaces_ip())

---
Для просмотра информации по соседним устройствам используется метод ```get_lldp_neighbors()``` или ```get_lldp_neighbors_detail()```

In [ ]:
pprint(leaf1.get_lldp_neighbors())

In [ ]:
pprint(leaf1.get_lldp_neighbors_detail())

---
Для получения информации по VRF используется метод ```get_network_instances()```

In [ ]:
pprint(leaf1.get_network_instances())

---
Для получения информации о конфигурации устройства используется метод ```get_config()``` с возможным указанием контейнера конфигурации.

In [ ]:
pprint(leaf1.get_config())

In [ ]:
pprint(leaf1.get_config('running'))

---
Для выполнения изменений конфигурации поддерживаются пять видов операций (имеются замечания при выполнении на оборудовании разных производителей):

 - Config.replace
 - Config.merge
 - Compare config
 - Atomic Changes
 - Rollback

---
Для выполнения команд в CLI используется метод ```cli()```, в качестве аргументов в который подается список строк-команд.

In [ ]:
pprint(leaf1.cli(['show running-config | json']))

---
Для применения изменений конфигурации используются методы ```load_merge_candidate()``` и ```load_replace_candidate()```. В качестве аргументов принимается либо конфигурация в виде строки, либо имя файла (если указаны оба аргумента, то файл имеет приоритет).

In [ ]:
leaf1.load_replace_candidate(filename='files/leaf1-lab.txt')

In [ ]:
pprint(leaf1.get_config('candidate'))

---
Для сравнения возможных изменений используется метод ```compare_config()```.

In [ ]:
print(leaf1.compare_config())

---
Для подтверждения изменений конфигурации используется метод ```commit_config()```.

In [ ]:
leaf1.commit_config()

In [ ]:
pprint(leaf1.get_config('candidate'))

In [ ]:
pprint(leaf1.get_config('running'))

---
Для отмены применения конфигурации используется метод ```rollback()```.

In [ ]:
leaf1.rollback()

In [ ]:
pprint(leaf1.get_config('candidate'))

In [ ]:
pprint(leaf1.get_config('running'))

---
Чтобы очистить контейнер `candidate` используется метод ```discard_config()```.

In [ ]:
leaf1.load_replace_candidate(filename='files/leaf1-lab.txt')

In [ ]:
pprint(leaf1.get_config('candidate'))

In [ ]:
leaf1.discard_config()

In [ ]:
pprint(leaf1.get_config('candidate'))

---
Для корректного завершения подключения к устройству рекомендуется использовать менеджер контекста с оператором ```with ... as ...```.

In [ ]:
with driver('172.20.20.11', 'admin', 'admin', optional_args = {'eos_autoComplete' : True}) as leaf1:
    pprint(leaf1.get_facts())
    pprint(leaf1.get_interfaces_ip())

---

### **Инструменты Python для работы с файлами конфигурации**

### **TextFSM**

In [24]:
!pip install textfsm

Defaulting to user installation because normal site-packages is not writeable


In [25]:
import textfsm
from pprint import pprint

---

In [26]:
with open('FSM/lldpneibs.txt', 'r') as f:
    lldp = f.read()

In [27]:
print(lldp[:1000])

Interface Ethernet1 detected 1 LLDP neighbors:

  Neighbor 5001.0054.f1d5/"Ethernet1", age 18 seconds
  Discovered 1 day, 1:02:33 ago; Last changed 1 day, 0:30:58 ago
  - Chassis ID type: MAC address (4)
    Chassis ID     : 5001.0054.f1d5
  - Port ID type: Interface name(5)
    Port ID     : "Ethernet1"
  - Time To Live: 120 seconds
  - System Name: "Leaf1.flane.ru"
  - System Capabilities : Bridge, Router
    Enabled Capabilities: Bridge, Router
  - Management Address Subtype: IPv4
    Management Address        : 10.0.0.1
    Interface Number Subtype  : ifIndex (2)
    Interface Number          : 5000000
    OID String                :
  - IEEE802.1 Port VLAN ID: 0
  - IEEE802.1/IEEE802.3 Link Aggregation
    Link Aggregation Status: Capable, Disabled (0x01)
    Port ID                : 0
  - IEEE802.3 Maximum Frame Size: 9236 bytes

Interface Ethernet2 detected 1 LLDP neighbors:

  Neighbor 5001.0041.ab3c/"Ethernet1", age 11 seconds
  Discovered 1 day, 1:02:33 ago; Last changed 22:5

In [28]:
with open('FSM/lldp_template.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseTextToDicts(lldp)

In [29]:
print(fsm.header)
pprint(result)

['LocalPort', 'MAC', 'RemotePort', 'FQDN', 'ManIP']
[{'FQDN': 'Leaf1.flane.ru',
  'LocalPort': 'Ethernet1',
  'MAC': '5001.0054.f1d5',
  'ManIP': '10.0.0.1',
  'RemotePort': 'Ethernet1'},
 {'FQDN': 'Leaf2.flane.ru',
  'LocalPort': 'Ethernet2',
  'MAC': '5001.0041.ab3c',
  'ManIP': '10.0.0.2',
  'RemotePort': 'Ethernet1'},
 {'FQDN': 'Leaf3.flane.ru',
  'LocalPort': 'Ethernet3',
  'MAC': '5001.0056.2a6e',
  'ManIP': '10.0.0.3',
  'RemotePort': 'Ethernet1'},
 {'FQDN': 'Leaf4.flane.ru',
  'LocalPort': 'Ethernet4',
  'MAC': '5001.00bb.2aef',
  'ManIP': '10.78.3.234',
  'RemotePort': 'Ethernet1'},
 {'FQDN': 'Leaf5.flane.ru',
  'LocalPort': 'Ethernet5',
  'MAC': '5001.002b.8d03',
  'ManIP': '10.0.0.5',
  'RemotePort': 'Ethernet1'},
 {'FQDN': 'Leaf6.flane.ru',
  'LocalPort': 'Ethernet6',
  'MAC': '5001.009c.5b8b',
  'ManIP': '10.0.0.6',
  'RemotePort': 'Ethernet1'},
 {'FQDN': 'Leaf7.flane.ru',
  'LocalPort': 'Ethernet7',
  'MAC': '5001.0014.9be4',
  'ManIP': '10.0.0.7',
  'RemotePort': 'Ethern

---

In [ ]:
with open('FSM/juniper_table_1.txt', 'r') as f:
    jun = f.read()
    f.close()
    
with open('FSM/juniper_template_1.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(jun)

print(fsm.header)
pprint(result)

In [ ]:
with open('FSM/juniper_template_2.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(jun)

print(fsm.header)
pprint(result)

---

In [ ]:
with open('FSM/juniper_table_2.txt', 'r') as f:
    data = f.read()
    f.close()
    
with open('FSM/juniper_template_3.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(data)

print(fsm.header)
pprint(result)

In [ ]:
with open('FSM/juniper_template_4.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(data)

print(fsm.header)
pprint(result)

In [ ]:
with open('FSM/juniper_template_5.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(data)

print(fsm.header)
pprint(result)

In [ ]:
with open('FSM/juniper_template_6.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(time)

print(fsm.header)
pprint(result)

---

In [ ]:
with open('FSM/route_table.txt', 'r') as f:
    data = f.read()
    f.close()
    
with open('FSM/route_template_1.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(data)

print(fsm.header)
pprint(result)

In [ ]:
with open('FSM/route_template_2.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(data)

print(fsm.header)
pprint(result)

In [ ]:
with open('FSM/route_template_3.txt') as template:
    fsm = textfsm.TextFSM(template)
    result = fsm.ParseText(data)

print(fsm.header)
pprint(result)

---

### **Jinja2**

In [ ]:
!pip install jinja2

In [30]:
from jinja2 import Template

In [ ]:
t = Template('Value = {{var}}')

print(t.render(var='Hello'))

In [ ]:
test = '{% for item in range(2, 40, 2) %}Value = {{ item }}\n{% endfor %}'

t = Template(test)

print(t.render())

In [ ]:
data = 11

test = '{% if var == 10 %} Hello {{ var }} {% endif %}'    

t = Template(test)

print(t.render(var=data))

In [31]:
from jinja2 import Environment, FileSystemLoader

In [32]:
env = Environment(loader=FileSystemLoader(['./jinja2']))

In [ ]:
t = env.get_template('for_template.j2')

In [ ]:
print(t.render())

In [ ]:
t = env.get_template('if_template.j2')

print(t.render(is_valid=True))
print(t.filename)

In [33]:
t = env.get_template('interfaces.j2')

In [37]:
t = env.get_template('interfaces.j2')

data = t.render(num = '101')

with open('jinja2/result.txt', 'w') as result:
    result.write(data)

---